### This part was used to reunite the json files from the Wikipedia Special Export into one csv file

In [1]:
import json
import csv
import pandas as pd
import numpy as np
import scipy.sparse as sp

files = {
    'Automobiles':'automobiles',
    'Biology':'biology',
    'Computing':'computing',
    'Design': 'design',
    'Economy': 'economy',
    'Electronics': 'electronics',
    'Engineering':'engineering',
    'Entertainment': 'entertainment',
    'Film':'film',
    'Geography & Places':'geography&places',
    'Mass Media':'massMedia',
    'Mathematics': 'mathematics',
    'Music':'music',
    'Sports':'sports'
}
content = {}
for key, value in files.iteritems():
    part1 = value + '_1.json'
    part2 = value + '_2.json'
    content[key] = '['
    with open(part1, 'r') as content_file:
        content[key] = content[key] + content_file.read().replace("\"}", "\"},")
    with open(part2, 'r') as content_file:
        content[key] = content[key] + content_file.read().replace("\"}", "\"},")
    content[key] = content[key][:-2] + ']'
    content[key] = content[key].replace('\n', ' ').rstrip()

In [3]:
finalJson = {}
auxStructure = []

for key, value in content.iteritems():
    jsonRows = json.loads(value)
    i = 0
    for row in jsonRows:
        i = i + 1
        if i == 10000:
            break
        row['category'] = key
        auxStructure.append(row)
    
finalJson = json.dumps(auxStructure)
    


In [2]:
auxStructure[0]
f = open('dataSet_Big.csv', 'w')
csv_file = csv.writer(f)
csv_file.writerow(["id", "Category", "Title", "Text"])
for item in auxStructure:
    csv_file.writerow([item['id'],item['category'], item['title'].encode('utf-8').replace('\n', ' ').rstrip(), item['text'].encode('utf-8').replace('\n', ' ').rstrip()])

f.close()

NameError: name 'auxStructure' is not defined

### Opening the resulting csv

In [2]:
dataSet = pd.read_csv('dataSet_big.csv')
#dataSet['Category_int'] = pd.Categorical(dataSet.Category)
#dataSet['Category_int'] = dataSet.Category_int.cat.codes
dataSet = dataSet.drop_duplicates(subset=['id'], keep=False)
print len(dataSet)
dataSet = dataSet.groupby(['Category']).head(6000)


104714


In [9]:
dataSet.groupby(['Category']).count()

,id,Title,Text
Category,,,
Automobiles,6000,6000,6000
Biology,6000,6000,6000
Computing,6000,6000,6000
Design,6000,6000,6000
Economy,6000,6000,6000
Electronics,6000,6000,6000
Engineering,6000,6000,6000
Entertainment,6000,6000,6000
Film,6000,6000,6000


### Apply the ML algorithm

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(dataSet['Text'], dataSet['Category'], random_state = 0, test_size=0.05)
clf = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')),
                 ('clf-svm', LinearSVC()),
                 ])
__ = clf.fit(X_train, y_train)


In [6]:
# Test precision

predicted = clf.predict(X_test)
np.mean(predicted == y_test)   

0.79937903033197999

In [25]:
# for key, value in files.iteritems():
#     predicted = clf
from sklearn.metrics import classification_report
print(classification_report(predicted, y_test, target_names=files.keys()))

                    precision    recall  f1-score   support

     Entertainment       0.95      0.93      0.94       297
            Sports       0.91      0.88      0.89       298
Geography & Places       0.70      0.69      0.70       321
             Music       0.64      0.71      0.67       280
       Mathematics       0.86      0.84      0.85       285
       Electronics       0.76      0.70      0.73       287
           Economy       0.70      0.75      0.72       303
           Biology       0.73      0.69      0.71       290
         Computing       0.90      0.82      0.86       334
       Automobiles       0.82      0.81      0.82       305
       Engineering       0.64      0.69      0.66       282
            Design       0.88      0.88      0.88       293
        Mass Media       0.85      0.87      0.86       315
              Film       0.87      0.91      0.89       297

       avg / total       0.80      0.80      0.80      4187



In [201]:
print(clf.predict(['Best guitar player']))

['Society & Social Sciences']


### Export model to use in Web Service

In [14]:
import pickle
pickle.dump(clf, open("model_big.pkl", 'wb'))

In [204]:
loaded_model = pickle.load(open("model.pkl", 'rb'))
print(loaded_model.predict(['Robot']))

['Engineering']


### Test different methods

In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(dataSet['Text'], dataSet['Category'], random_state = 0, test_size=0.15)
pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('clf-svm', LinearSVC()),
                 ])
vectorizer = [TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')]
algorithms = [GaussianNB(), AdaBoostClassifier(), DecisionTreeClassifier(3), LinearSVC()]
params = [
    {
        'tfidf': vectorizer,
        'clf-svm': algorithms
    },
]

grid = GridSearchCV(pipe, cv=2, n_jobs=1, param_grid=params)
grid.fit(X_train, y_train)

mean_scores = np.array(grid.cv_results_['mean_test_score'])

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(dataSet['Text'], dataSet['Category'], random_state = 0, test_size=0.05)
clf = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')),
                 ('clf-svm', LinearSVC()),
                 ])
__ = clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)  

0.79937903033197999

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')),
                 ('clf-svm', MultinomialNB()),
                 ])
__ = clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)  

0.71865297348937185

In [11]:
clf = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')),
                 ('clf-svm', AdaBoostClassifier()),
                 ])
__ = clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)  

0.51182230714115118

In [12]:
clf = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')),
                 ('clf-svm', DecisionTreeClassifier(3)),
                 ])
__ = clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)  


KeyError: 3

In [18]:
clf = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')),
                 ('clf-svm', KNeighborsClassifier(3)),
                 ])
__ = clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)  

0.10460950561261047